# In Depth Experiment Configuration

The experiment class provides an interface that you can manage your experiment with backward compatibility. It means that even your Experiment has been built/defined you will be able to configure its parameters. This feature will provide more control over your experiment even after your running your experiment for several rounds. In this tutorial, detailed experiment interface will be explained using MNIST basic example.

## Configuring Environment
Before running this notebook, you need to configure your environment by completing following steps:

### Starting the Network Component
Please run following command to start Network component that provided communication between your notebook and the node;
```shell
{FEDBIOMED_DIR}/scripts/fedbiomed_run network
```
<div class="note">
<p>This command will launch docker containers. Therefore, please make sure that your Docker engine is up and running.</p>
</div>

### Deploying MNIST Dataset in the None
Please run following command to add MNIST dataset into your Node. This command will deploy MNIST dataset in your default node whose config file is located in `{FEDBIOMED_DIR}/etc` directory as `config_node.ini`

After running following command, please select data type `2) default`, use default `tags` and select the folder where MNIST dataset will be saved.

```shell
{FEDBIOMED_DIR}/scripts/fedbiomed_run node add
```

### Starting the Node
 After you have successfully completed previous step, please run following command to start your node.

```shell
{FEDBIOMED_DIR}/scripts/fedbiomed_run node start
```

## Creating a Training Plan

Before declaring an experiment, the training plan that will be used for federated training should be defined. The model that is goıng to be used is exactly the same training plan that has been created in the Basic MNIST tutorial. We recommend you to follow Basic MNIST tutorial on PyTorch Framework to understand following steps.

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms


# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    
    # Defines and return model 
    def init_model(self, model_args):
        return self.Net(model_args = model_args)
    
    # Defines and return optimizer
    def init_optimizer(self, optimizer_args):
        return torch.optim.Adam(self.model().parameters(), lr = optimizer_args["lr"])
    
    # Declares and return dependencies
    def init_dependencies(self):
        deps = ["from torchvision import datasets, transforms"]
        return deps
    
    class Net(nn.Module):
        def __init__(self, model_args):
            super().__init__()
            self.conv1 = nn.Conv2d(1, 32, 3, 1)
            self.conv2 = nn.Conv2d(32, 64, 3, 1)
            self.dropout1 = nn.Dropout(0.25)
            self.dropout2 = nn.Dropout(0.5)
            self.fc1 = nn.Linear(9216, 128)
            self.fc2 = nn.Linear(128, 10)

        def forward(self, x):
            x = self.conv1(x)
            x = F.relu(x)
            x = self.conv2(x)
            x = F.relu(x)
            x = F.max_pool2d(x, 2)
            x = self.dropout1(x)
            x = torch.flatten(x, 1)
            x = self.fc1(x)
            x = F.relu(x)
            x = self.dropout2(x)
            x = self.fc2(x)


            output = F.log_softmax(x, dim=1)
            return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.model().forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


## Creating an Experiment Step by Step  

The experiment class can be created without passing any argument. This will just build an empty experiment object. Afterwards, you will be able to define your arguments using setters of the experiment object.


<div class="note"><p>It is always possible to create a fully configured experiment by passing all arguments during the initialization. You can also create your experiment with some of the arguments and set the other arguments after.</p></div>

### Building an Empty Experiment


After building an empty experiment you won't be able to perform federated training, since it is not fully configured. That's why the output of the initialization will always remind you that the experiment is not fully configured.

In [2]:
from fedbiomed.researcher.experiment import Experiment
exp = Experiment()

2023-02-02 14:14:39,089 fedbiomed INFO - Messaging researcher_26940cf7-f022-46b4-84ad-2a55a4c35780 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7fa6ee83b040>
2023-02-02 14:14:39,111 fedbiomed DEBUG - Experiment not fully configured yet: no training data
2023-02-02 14:14:39,112 fedbiomed DEBUG - Experiment not fully configured yet: no node selection strategy
2023-02-02 14:14:39,113 fedbiomed DEBUG - Experiment not fully configured yet: no valid training plan, training_plan_class=None training_plan_class_path=None
2023-02-02 14:14:39,114 fedbiomed DEBUG - Experiment not fully configured yet: no valid training plan, training_plan=None training_plan_path=None
2023-02-02 14:14:39,114 fedbiomed DEBUG - Experiment not fully configured yet: no job. Missing proper training plan definition (training_plan=None training_plan_path=None)


### Displaying Current Status of Experiment
As an addition to output of the initialization, to find out more about the current status of the experiment, you can call the `info()` method of your experiment object. This method will print the information about your experiment and what you should complete to be able to start your federated training.

In [ ]:
exp.info()

Based on the output, some arguments are defined with default values, while others are not. Model arguments, training arguments, tags, round limit, training data etc. have no default value, and they are required to be set. However, these arguments are related to each other. For example, to be able to define your federated training data you need to define the `tags` first, and then while setting your training data argument, experiment will be able to send search request to the nodes to receive information about the datasets. These relations between the arguments will be explained in the following steps.

### Setting Training Plan for The Experiment

The training plan that is going to be used for training can be set in the experiment using the method `set_training_plan_class`.

In [3]:
exp.set_training_plan_class(training_plan_class=MyTrainingPlan)


__main__.MyTrainingPlan

### Setting The Training Plan path (Special Case)
The `set_training_plan_path` is used in the case where you don't define the training plan directly in the notebook, but rather import it from a python module file. In this case, the `training_plan_file` argument is used to specify the module file's directory. However, the experiment also needs to know your class name. You specify your class name as a **`string`** with `set_training_plan_class` method. Thus the backend knows which class to use in the module file.

```python
exp.set_training_plan_path(training_plan_path='path/to/your/script.py')
exp.set_training_plan_class(training_plan_class='TrainingPlanClassAsString')
```

### Setting Model and Training Arguments
In the previous step, the model has been defined for your experiment. Now, you can define your model arguments and training arguments that will be used respectively for building your model class and training your model on the node side. The methods `set_model_args` and `set_training_args` of the experiment class will allow you to set these arguments.

<div class="">
    <p>There isn't any requirement on the order of defining model class and mode/training arguments. It is also possible to
        define model/training arguments first and model class after. 
    </p>    
<div>


In [4]:
# Model arguments should be an empty Dict, since our model does not require 
# any argument for initialization
model_args = {}

# Training Arguments
training_args = {
    'batch_size': 48, 
    'optimizer_args': {
        'lr': 1e-3, 
    },
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

exp.set_model_args(model_args=model_args)
exp.set_training_args(training_args=training_args)

{'batch_size': 48,
 'optimizer_args': {'lr': 0.001},
 'epochs': 1,
 'dry_run': False,
 'batch_maxnum': 100,
 'num_updates': None,
 'test_ratio': 0.0,
 'test_on_local_updates': False,
 'test_on_global_updates': False,
 'test_metric': None,
 'test_metric_args': {},
 'log_interval': 10,
 'fedprox_mu': None,
 'use_gpu': False,
 'dp_args': None}

### Setting Tags

The tags for the dataset search request can be set using `set_tags` method of experiment object. 

<br><div class="note"><p>Setting tags does not mean sending dataset search request. Search request is sent while setting training data. `tags` is the argument that is required for the search request.</p></div>

The arguments `tags` of `set_tags` method should be an array of tags which are in `string` type or just a tag in `string` type.

In [5]:
tags = ['#MNIST', '#dataset']
exp.set_tags(tags = tags)

['#MNIST', '#dataset']

To see the tags that are set, you can run `tags()` method of experiment object. 

In [6]:
exp.tags()

['#MNIST', '#dataset']

### Setting Nodes
The `nodes` arguments indicates the nodes that are going to be used for the experiment. By default, it is equal to `None` which means every node up and running will be part of the experiment as long as they have the dataset that is going to be used for training. If the `nodes` has been set in advance, the search request for the dataset search will be sent only the nodes that are indicated. You can set nodes using the method `set_nodes(noes=nodes)`. This method takes `nodes` argument which should be an array of node ids which are in `string` type or just a single node id as `string`.

Since the node ids can change randomly, to make this notebook runnable in all environments, we won't be setting nodes for the experiment.


### Setting Training Data
Training data is a `FederatedDataset` instance which comes from the module `fedbiomed.researcher.datasets`. There are several ways to define your training data.

1. You can run `set_training_data(training_data=None, from_tags=True)`. This will send search request to the nodes to get dataset information by using the `tags` which are defined before.
2. You can provide `training_data` argument which is an instance of `FederatedDataSet`. 
3. You can provide `training_data` argument as python `dict` and setter will create a `FederatedDataSet` object by itself.

<div class="note"><p>While using the last option please make sure that your `dict` object is configured as coherent to `FederatedDataSet` schema. Otherwise, you might get error while running your experiment. </p></div>

If you run `set_training_data(training_data=None)`. No training data is defined yet for the experiment (`training_data` is set to `None`).


In [9]:
training_data = exp.set_training_data(training_data=None, from_tags=True)

2023-02-02 14:15:26,591 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2023-02-02 14:15:36,605 fedbiomed INFO - Node selected for training -> node_2ff3675d-9d8b-458c-aa5a-6b44ecfe84d2
2023-02-02 14:15:36,607 fedbiomed DEBUG - Training data changed, you may need to update `aggregator`


Since it will send search request to the nodes, the output will inform you about selected nodes for training. It means that those nodes have the dataset and able to train your model.

`set_training_data` will return a `FederatedDataSet` object. You can either use the return value of the setter or the getter for training data which is `training_data()`.

In [10]:
training_data = exp.training_data()

To inspect the result in detail you can call the method `data()` of the `FederatedDataSet` object. This will return a python dictionary that includes information about the datasets that has been found in the nodes. 

In [11]:
training_data.data()

{'node_2ff3675d-9d8b-458c-aa5a-6b44ecfe84d2': [{'name': 'MNIST',
   'data_type': 'default',
   'tags': ['#MNIST', '#dataset'],
   'description': 'MNIST database',
   'shape': [60000, 1, 28, 28],
   'dataset_id': 'dataset_50601359-8b47-4694-aae8-cad88be36c4d',
   'dtypes': [],
   'dataset_parameters': None}]}

In [15]:
from fedbiomed.researcher.requests import Requests
req = Requests()
data = req.search(['#MNIST'])

2023-02-02 14:18:26,806 fedbiomed INFO - Searching dataset with data tags: ['#MNIST'] for all nodes
2023-02-02 14:18:36,814 fedbiomed INFO - Node selected for training -> node_2ff3675d-9d8b-458c-aa5a-6b44ecfe84d2


In [16]:
print(data)
type(data)

{'node_2ff3675d-9d8b-458c-aa5a-6b44ecfe84d2': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28], 'dataset_id': 'dataset_50601359-8b47-4694-aae8-cad88be36c4d', 'dtypes': [], 'dataset_parameters': None}]}


dict

In [18]:
from fedbiomed.researcher.datasets import FederatedDataSet
f = FederatedDataSet(data)

In [23]:
f.shapes()

{'node_2ff3675d-9d8b-458c-aa5a-6b44ecfe84d2': 60000}

2023-02-02 16:09:29,869 fedbiomed INFO - ERROR
					 NODE test_logger_node_51ab1388-7e79-450f-82f5-b3efafca319e
					 MESSAGE: mqtt+console ERROR message
-----------------------------------------------------------------
2023-02-02 16:09:40,506 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Messaging mock_researcher__res_5dbd1317-667d-471e-a1a5-85353d176695_XXX successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f1387488670>
-----------------------------------------------------------------
2023-02-02 16:09:42,551 fedbiomed INFO - WARNING
					 NODE node_1234
					 MESSAGE: Target data seems to be a regression, metric ACCURACY might not be appropriate
-----------------------------------------------------------------
2023-02-02 16:09:42,552 fedbiomed INFO - WARNING
					 NODE node_1234
					 MESSAGE: Target data seems to be a regression, metric ACCURACY might not be appropriate
-----------------------------------------------

2023-02-02 16:09:42,630 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `weighted` calculation for the metric PRECISION
-----------------------------------------------------------------
2023-02-02 16:09:42,633 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `samples` calculation for the metric F1_SCORE
-----------------------------------------------------------------
2023-02-02 16:09:42,636 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `samples` calculation for the metric RECALL
-----------------------------------------------------------------
2023-02-02 16:09:42,639 fedbiomed INFO - INFO
					 NODE node_1234
					 MESSAGE: Actual/True values (y_true) has more than two levels, using multiclass `samples` calculation for the metric PRECISION
-----

2023-02-02 16:09:42,810 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: ErrorNumbers.FB318: bad secure aggregation request received by mock_node__nod_05a69e94-88d3-4105-9179-868de4c0d697_XXX: FB318: Secure aggregation setup error: bad parameter `parties` : ['party1', 'party2'] : need  at least 3 parties for secure aggregation
-----------------------------------------------------------------
2023-02-02 16:09:42,810 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: ErrorNumbers.FB318: bad secure aggregation request received by mock_node__nod_05a69e94-88d3-4105-9179-868de4c0d697_XXX: 
-----------------------------------------------------------------
2023-02-02 16:09:42,811 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: ErrorNumbers.FB318: bad secure aggregation request received by mock_node__nod_05a69e94-88d3-4105-9179-868de4c0d697_XXX: 
-----------------------------------------------------------------
2023-02-02 16:09:42,815 fedbiomed INFO - INFO
					 NOD

2023-02-02 16:09:42,871 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB201: server not reachable : CALLABLE_RAISING_TOO_MANY_REDIRECTION_ERRORS HTTP request exceeds max number of redirection
-----------------------------------------------------------------
2023-02-02 16:09:42,872 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB604: repository error : bad URL when issuing a unknwon HTTP request a/file/path(details :Invalid URL 'a/file/to/upload': No scheme supplied. Perhaps you meant https://a/file/to/upload? )
-----------------------------------------------------------------
2023-02-02 16:09:43,100 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB201: server not reachable when issuing a unknwon HTTP request a/file/path to http://a.fake.url: name or service not known
-----------------------------------------------------------------
2023-02-02 16:09:43,101 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB200: undetermined repository serv

...

2023-02-02 16:15:32,924 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB318: Secure aggregation setup error: bad parameter `secagg_id` should not be empty string
-----------------------------------------------------------------
2023-02-02 16:15:32,925 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB318: Secure aggregation setup error: bad parameter `['not a string']` should be a <class 'str'>: wrong input: ['not a string'] should be a <class 'str'>
-----------------------------------------------------------------
2023-02-02 16:15:32,926 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB318: Secure aggregation setup error: bad parameter `999` should be a <class 'str'>: wrong input: 999 should be a <class 'str'>
-----------------------------------------------------------------
2023-02-02 16:15:32,926 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB318: Secure aggregation setup error: bad parameter `job_id` must be an empty string
--------

2023-02-02 16:15:32,937 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB415: secure aggregation handling error: bad parameter `parties` must be a list of strings: wrong input: 3 should be a <class 'str'>
-----------------------------------------------------------------
2023-02-02 16:15:32,938 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB415: secure aggregation handling error: bad parameter `parties` must be a list of strings: wrong input: ['party3'] should be a <class 'str'>
-----------------------------------------------------------------
2023-02-02 16:15:32,938 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB415: secure aggregation handling error: bad parameter `parties` must be a list of strings: wrong input: ['party3'] should be a <class 'str'>
-----------------------------------------------------------------
2023-02-02 16:15:32,939 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB415: secure aggregation handling error: bad para

2023-02-02 16:15:38,402 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB415: secure aggregation handling error: some parties did not answer before timeout ['party2', 'party3']
-----------------------------------------------------------------
2023-02-02 16:15:39,405 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB415: secure aggregation handling error: some parties did not answer before timeout ['party2', 'party3']
-----------------------------------------------------------------
2023-02-02 16:15:42,409 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB415: secure aggregation handling error: some parties did not answer before timeout ['party2', 'party3']
-----------------------------------------------------------------
2023-02-02 16:15:43,412 fedbiomed INFO - ERROR
					 NODE node_1234
					 MESSAGE: FB415: secure aggregation handling error: some parties did not answer before timeout ['party2', 'party3']
----------------------------------------------

2023-02-02 16:15:46,505 fedbiomed INFO - CRITICAL
					 NODE node_1234
					 MESSAGE: FB605: training plan error: An error occurred while computing the ACCURACY metric: 
-----------------------------------------------------------------
2023-02-02 16:15:46,506 fedbiomed INFO - CRITICAL
					 NODE node_1234
					 MESSAGE: FB605: training plan error: An error occurred while computing the Custom metric: 
-----------------------------------------------------------------
2023-02-02 16:15:46,509 fedbiomed INFO - CRITICAL
					 NODE node_1234
					 MESSAGE: FB605: training plan error: metric value should be one of type int, float, numpy scalar, numpy.ndarray, torch.Tensor, or list or dict wrapping such values; but received <class 'NoneType'>
-----------------------------------------------------------------
2023-02-02 16:15:46,604 fedbiomed INFO - CRITICAL
					 NODE node_1234
					 MESSAGE: FB414: bad type or value for training arguments: The key `fedprox_mu` does not exist in training args
--

2023-02-02 16:15:55,296 fedbiomed INFO - CRITICAL
					 NODE node_1234
					 MESSAGE: FB606: model manager error: search request on database failed. Details: Expecting value: line 1 column 1 (char 0)
-----------------------------------------------------------------
2023-02-02 16:15:55,297 fedbiomed INFO - CRITICAL
					 NODE node_1234
					 MESSAGE: FB606: model manager error: search request on database failed. Details: Expecting value: line 1 column 1 (char 0)
-----------------------------------------------------------------
2023-02-02 16:15:55,298 fedbiomed INFO - CRITICAL
					 NODE node_1234
					 MESSAGE: FB606: model manager error: search request on database failed. Details: Expecting value: line 1 column 1 (char 0)
-----------------------------------------------------------------
2023-02-02 16:15:55,298 fedbiomed INFO - CRITICAL
					 NODE node_1234
					 MESSAGE: FB606: model manager error: search request on database failed. Details: Expecting value: line 1 column 1 (char 0)
--

As it is mentioned before, setting training data once doesn't mean that you can't change it. You can create a new `FederatedDataSet` with a `dict` that includes the information about the datasets. This will allow you to select the datasets that will be used for federated training.

<div class="note"><p>Since the dataset information will be provided, there will be no need to send request to the nodes</p></div>

In [ ]:
from fedbiomed.researcher.datasets import FederatedDataSet 

tr_data = training_data.data()
federated_dataset = FederatedDataSet(tr_data)
exp.set_training_data(training_data = federated_dataset)

Or, you can directly use `tr_data` in `set_training_data()`

In [ ]:
exp.set_training_data(training_data = tr_data)

<div class="note">
    <p>
        If you change the tags for the dataset by using <code>set_tags</code> and if there is already a defined training data in your experiment object, you have to update your training data by running <code>exp.set_training_data(training_data=None)</code>.  
    </p>
</div>

### Setting an Aggregator  

An aggregator is one of the required arguments for the experiment. It is used for aggregating model parameters that are received from the nodes after every round. By default, when the experiment is initialized without passing any aggregator, it will automatically use the default `FedAverage` aggregator class. However, it is also possible to set a different aggregation algorithm with the method `set_aggregator`. Currently, Fed-BioMed has only `FedAverage` but it is possible to create a custom aggregator classes.

You can see the current aggregator by running `exp.aggregator()`. It will return the aggregator object that will be used for aggregation. 

In [ ]:
exp.aggregator()

If we supposed that you have created your own aggregator, you can set it as follows,

In [ ]:
from fedbiomed.researcher.aggregators.fedavg import FedAverage
exp.set_aggregator(aggregator=FedAverage)

If your aggregator class needs initialization parameters, you can build your class and pass as an object .

In [ ]:
fed_average = FedAverage()
exp.set_aggregator(aggregator=fed_average)

### Setting Node Selection Strategy

Node selection Strategy is also one of the required arguments for the experiment. It is used for selecting nodes before each round of training. Since the strategy will be used for selecting nodes, before setting the strategy, training data should be already set. Then, strategy will be able to which nodes are current with their dataset.

By default, `set_strategy(node_selection_strategy=None)` will use the default `DefaultStrategy` class. It is default strategy that selects all the nodes available with their datasets at the moment. However, it is also possible to set different strategies. Currently, Fed-BioMed has only `DefaultStrategy` but you can create your custom strategy classes.



In [ ]:
exp.set_strategy(node_selection_strategy=None)

Or, you can directly pass `DefaultStrategy`

In [ ]:
from fedbiomed.researcher.strategies.default_strategy import DefaultStrategy
exp.set_strategy(node_selection_strategy=DefaultStrategy)

# To make the strategy has been set
exp.strategy()

### Setting Round Limit

Round limit is the limit that indicates max number of rounds of the training. By default, it is `None` and it needs to be set before running your experiment. You can set the round limit with the method `set_round_limit`. Round limit can  be changed after running one or several rounds of training. You can always execute `exp.round_limit()` to see current round limit.

In [ ]:
exp.set_round_limit(round_limit=2)
exp.round_limit()

### Setting Job to Manage Federated Training Rounds

Job is a class that manages federated training rounds. Before setting job, strategy for selecting nodes, model and training data should be set. Therefore, please make sure that they all defined before setting job.  The method `set_job` creates the Job instance and it does not take any argument. 

In [ ]:
exp.set_job()
exp.job()

### Setting Secure Aggregation

Secure aggregation enables nodes to send encrypted updates to the researcher. Thus researcher cannot read cleartext updates from the nodes. Nevertheless, researcher can aggregate updates from the nodes. The aggregated model parameters are then in cleartext, and the researcher can read them.

The method `use_secagg` toggles usage of secure aggregation. When `True`, a secure aggregation context (cryptographic material) for the experiment is negotiated between the experiment parties (if not existing yet) or existing secure aggregation context is re-used. Then usage of secure aggregation by the experiment is activated.

When negotiating a secagg context, `secagg_timeout` is the maximum setup time for the negotiation of each secure aggregation context element (server key and biprime). Thus maximum negotiation time is twice `secagg_timeout`.
Secure aggregation needs at least 2 active nodes in the experiment (thus 3 parties including the researcher).

In [ ]:
exp.set_use_secagg(use_secagg=True, timeout=10)

In [ ]:
print("Using secagg: ", exp.use_secagg())
exp_servkey, exp_biprime = exp.secagg_context()
if exp_servkey:
    print(f"Secagg servkey:\n- status {exp_servkey.status()}\n- secagg_id {exp_servkey.secagg_id ()}" \
        f"\n- context {exp_servkey.context()}")
else:
    print("No secagg servkey")
if exp_biprime:
    print(f"Secagg biprime:\n- status {exp_biprime.status()}\n- secagg_id {exp_biprime.secagg_id ()}" \
        f"\n- context {exp_biprime.context()}")
else:
    print("No secagg biprime")

In [ ]:
exp.set_use_secagg(False)

### Controlling Experiment Status Before Starting Training Rounds
Now, let's see if our experiment is ready for the training.

In [ ]:
exp.info()

If the experiment is ready, you will see the message that says `Experiment can be run now (fully defined)` at the bottom of the output. So now, we can run the experiment

## Running The Experiment

As long as `info()` says that the experiment is fully defined you will be able to run your experiment. Experiment has two methods  as `run()` and `run_once()` for running training rounds.

 - `run()` runs the experiment rounds from current round to round limit. If the round limit is reached it will indicate that the round limit has been reach. However, the method `run` takes to arguments as `round` and `increase`. 
    - `round` is an integer that indicates number of rounds that are going to be run. If the experiment is at round `0`, the round limit is `4`, and if you pass `round` as 3, it will run the experiment only for `3` rounds.
    - `increase` is a boolean that indicates whether round limit should be increased if the given `round` pass over the round limit. For example, if the current round is `3`, the round limit is `4`, and the `round` argument is `2`, the experiment will increase round limit to `5`
    
 - `run_once()` runs the experiment for single round of training. If the round limit is reached it will indicate that the round limit has been reach. However, if it is executed as `run_once(increase=True)` when the round limit is reach, it increases the round limit for one round.

In [ ]:
exp.run_once()

After running the experiment for once, you can check the current round. It returns `1` which means only one round has been run.

In [ ]:
exp.round_current()

Now, let's run the experiment with `run_once()` again. 

In [ ]:
exp.run_once()

Since the round limit has been set to `2` the round limit had been reached. If you try to run `run()` or `run_once()` the experiment will indicate that the round limit has been reached.

In [ ]:
exp.run_once()

In [ ]:
exp.run()

After this point, if you would like to run the experiment you can increase round limit with `set_round_limit(round)`

In [ ]:
exp.set_round_limit(4)
print('Round Limit    : ' , exp.round_limit())
print('Current Round  : ' , exp.round_current())

The round limit of the experiment has been set to `4` and the completed number of rounds is `2`. It means if you run the experiment with method `run()` without passing any argument, it will run the experiment for `2` rounds.

In [ ]:
exp.run()

Let's check the current round status of the experiment. 

In [ ]:
print('Round Limit    : ' , exp.round_limit())
print('Current Round  : ' , exp.round_current())

Another way to run our experiment if the round limit is reached is passing `rounds` to the method `run()`. For example, following cell will run the experiment for `2` more rounds.

In [ ]:
exp.run(rounds=2, increase=True) # increase is True by default

If the argument `increase` is `False`, it will not increase the round limit automatically. 

In [ ]:
exp.run(rounds=2, increase=False)

In [ ]:
print('Round Limit    : ' , exp.round_limit())
print('Current Round  : ' , exp.round_current())

It is also possible to increase number of rounds while running the experiment with `run_once()` by passing `increase` argument as `True`

In [ ]:
exp.run_once(increase=True)

In [ ]:
print('Round Limit    : ' , exp.round_limit())
print('Current Round  : ' , exp.round_current())

### Changing Training Arguments for the Next Round

The method `set_training_args()` allows you to change the training arguments even you've already run your experiment several times. Thanks to the method `set_training_args()` you will be able to configure your training from one round to another. For example, we can change our `batch_size` to `64` and `batch_maxnum` to `50` for the next round.


In [ ]:
# Training Arguments
training_args = {
    'batch_size': 64, 
    'optimizer_args': {
        'lr': 1e-3
    },
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 50
}

exp.set_training_args(training_args=training_args)

In [ ]:
exp.run_once(increase=True)

### Conclusions 
The experiment class is the interface and the orchestrator of the whole processes behind federated training on the researcher side. It allows you to manage your federated training experiment easily. It has been extended with setter and getter methods to ease its declaration. This also provides more control before, during or after the training rounds. The purpose of the experiment class is to provide a robust interface for end-user to make them able to easily perform their federated training on Fed-BioMed nodes.